In [ ]:
USE master
GO
IF NOT EXISTS(SELECT * FROM sys.databases WHERE name = 'running')
    CREATE DATABASE running

In [ ]:
USE running

In [ ]:
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://controller-svc/default');

In [ ]:
IF NOT EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'parquet_file')
    CREATE EXTERNAL FILE FORMAT parquet_file
    WITH (
        FORMAT_TYPE = PARQUET
    );

In [ ]:
IF EXISTS(SELECT * FROM sys.external_tables WHERE name = 'activities')
    DROP EXTERNAL TABLE activities

In [ ]:
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'activities')
    CREATE EXTERNAL TABLE [activities]
    ("Runner" VARCHAR(50) ,
    "DTE" Date ,
    "Activity" VARCHAR(50) COLLATE LATIN1_GENERAL_100_CI_AS_SC_UTF8,
    "Distance" float ,
    "Duration" float)
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/running/parquet',
        FILE_FORMAT = parquet_file
    );

In [ ]:
SELECT TOP 5 * from activities

In [ ]:
SELECT DATEPART(year, DTE)* 100+DATEPART(month, DTE) MT, round(SUM(Distance / 1000),1) KM
FROM activities
WHERE DTE>='01.01.2019'
GROUP BY DATEPART(year, DTE)* 100+DATEPART(month, DTE)
ORDER BY DATEPART(year, DTE)* 100+DATEPART(month, DTE)